# Data Exploration for Client Recommendation

Before doing client recommendation, I'm going to do some data analysis to see whether my plan is practical.

In [1]:
import pandas as pd
import numpy as np

In [5]:
all_order_train = pd.read_pickle('../all_order_train.pkl')
all_order_test = pd.read_pickle('../all_order_test.pkl')
print(all_order_train.shape, all_order_test.shape)

all_order_train.head()

(33720820, 12) (98286, 12)


order_id  user_id  order_number  order_hour_of_day  product_id  \
0   2539329        1             1                  8         196   
1   2231262       31            17                 11         196   
2   3058369      195            34                 10         196   
3   2257155      951             3                 20         196   
4   1121647      992             7                 10         196   

  purchase_date                              merchant product_name  price  \
0    2019-02-13  L&M Markets (Hometown Grocers Co-op)         Soda   3.08   
1    2019-03-10  L&M Markets (Hometown Grocers Co-op)         Soda   3.08   
2    2019-02-25  L&M Markets (Hometown Grocers Co-op)         Soda   3.08   
3    2019-03-09  L&M Markets (Hometown Grocers Co-op)         Soda   3.08   
4    2019-03-08  L&M Markets (Hometown Grocers Co-op)         Soda   3.08   

         aisle department  week_number  
0  soft drinks  beverages            7  
1  soft drinks  beverages           10  
2  soft drinks  beverages            9  
3  soft drinks  beverages           10  
4  soft drinks  beverages           10

## Shared Functions

In [18]:
def get_percentile(col):
    result = {'min': np.percentile(col, 0), '1%':np.percentile(col, 1),
             '5%':np.percentile(col, 5), '15%':np.percentile(col, 15),
             '25%':np.percentile(col, 25), '50%':np.percentile(col, 50), '75%':np.percentile(col, 75),
             '85%':np.percentile(col, 85), '95%':np.percentile(col, 95), '99%':np.percentile(col, 99),
              'max':np.percentile(col, 100)}
    return result

## Client Appearance Distribution

In [10]:
client_weeks_df = all_order_train[['user_id', 'week_number']].astype('str').drop_duplicates()\
          .groupby(['user_id'], as_index=False)['week_number']\
          .agg(['count']).reset_index()\
          .sort_values(['count'], ascending=False)

client_weeks_df.head()

user_id  count
126573   28325      6
135599   36449      6
172385   69557      6
81882   173692      6
9358     10842      6

In [12]:
client_weeks_df['count'].value_counts()

5    61256
4    61174
3    51166
2    20929
6    10836
1      848
Name: count, dtype: int64

## Client Purchase Ability to Merchant Avg Clients

* `merchant_match_score = abs(1 - target_client_avg_purchase_dollars / merchannt_avg_sales_per_client)`
  * This score will be used to find merchant candidates in collaborative filtering
  * Smaller the score, the better match

In [16]:
merchant_sales_df = all_order_train[['merchant', 'user_id', 'price']].drop_duplicates()\
          .groupby(['merchant'], as_index=False)\
          .agg({'user_id': 'count', 'price': 'sum'})

merchant_sales_df.head()

merchant  user_id         price
0       49th Parallel Grocery  1211766  1.452954e+07
1           Asian Food Centre  1210187  1.451086e+07
2               Askew's Foods  1203583  1.443146e+07
3  Avril (Health Supermarket)  1200036  1.439485e+07
4               Calgary Co-op  1017673  1.218554e+07

In [17]:
merchant_sales_df['avg_sales_per_client'] = round(merchant_sales_df['price']/merchant_sales_df['user_id'], 4)

merchant_sales_df.head()

merchant  user_id         price  avg_sales_per_client
0       49th Parallel Grocery  1211766  1.452954e+07               11.9904
1           Asian Food Centre  1210187  1.451086e+07               11.9906
2               Askew's Foods  1203583  1.443146e+07               11.9904
3  Avril (Health Supermarket)  1200036  1.439485e+07               11.9953
4               Calgary Co-op  1017673  1.218554e+07               11.9739

In [19]:
get_percentile(merchant_sales_df['avg_sales_per_client'])

{'min': 11.8612,
 '1%': 11.87368,
 '5%': 11.907,
 '15%': 11.94756,
 '25%': 11.9562,
 '50%': 11.9693,
 '75%': 11.9822,
 '85%': 11.986419999999999,
 '95%': 11.99046,
 '99%': 11.993044000000001,
 'max': 11.9953}

* Seems all the merchants have similar average sales per client... Since I was using uniform random distribution...

In [20]:
client_purchase_df = all_order_train[['user_id', 'price', 'order_id']].drop_duplicates()\
          .groupby(['user_id'], as_index=False)\
          .agg({'price': 'sum', 'order_id': 'count'})

client_purchase_df.head()

user_id    price  order_id
0        1   560.43        70
1        2  2685.71       226
2        3   994.48        88
3        4   282.01        18
4        5   543.37        46

In [21]:
client_purchase_df['avg_purchase'] = round(client_purchase_df['price']/client_purchase_df['order_id'], 4)

client_purchase_df.head()

user_id    price  order_id  avg_purchase
0        1   560.43        70        8.0061
1        2  2685.71       226       11.8837
2        3   994.48        88       11.3009
3        4   282.01        18       15.6672
4        5   543.37        46       11.8124

In [22]:
get_percentile(client_purchase_df['avg_purchase'])

{'min': 2.01,
 '1%': 5.964432,
 '5%': 7.968640000000001,
 '15%': 9.576039999999999,
 '25%': 10.4277,
 '50%': 11.8533,
 '75%': 13.2933,
 '85%': 14.14598,
 '95%': 15.9362,
 '99%': 19.205927999999982,
 'max': 54.91}

* So the `merchant_match_score` distribution will be similar to this.